For this Notebook to work correctly, the virtual memory on the Windows 10 machine where this Notebook is being run must be increased. This is because the notebook will download and process satellite images that are, generally, very large. In order to achieve this, there are some system modifications that need to be done before running the Notebook. To increase the virtual memory of the machine:

Go to Control panel---> System---> Advanced System Settings.

In the window that appears select the Advance tab, then settings, in the new window select the advanced tab, the click on change. Here, untick the "Automatically manage paging file for all drives" Select then the Custom size and in the initial size (MB) box. Enter 4096 and in the Maximum size (MB) box enter 7000 or 8000. (The maximum size can be increased if the mosaic to be created is very large).

In [1]:
#Importing the neccesary libraries for downloading data 
import requests
import time
from datetime import datetime
import pandas as pd
from pathlib import Path
import xmltodict
from zipfile import ZipFile

The following cell will create a folder, where the notebook is located, and all
the downloaded images will be stored there 

In [2]:
# get input for "test name" eg 'PRD_STANDARDISED_TEST_WPS_S1x10_RUN1'
RUNNAME = input()

APITEST130721


In [3]:
output_fmt='%Y%m%dT%H%M%S'
pretty_fmt='%Y-%m-%d %H:%M:%S'
headers = {'Content-type': 'application/xml','User-Agent': 'curl/7.65.1'}
# need to download gsdownload-template.xml to scripts/xml folder.
wps_test_config = {
        'gs:Download':{
            'template_xml':'gsdownload-template.xml',
            'mimetype':'application/zip',
            'output':'result.zip',
        }}

At this point it must be noted that this script in order to work, the input images must be of the same granule from different time stamps. This script will not create a mosaic of diferent granules but a mosaic of the same granule without any cloud coverage using diferent sensing dates of the same granule.  

In [4]:
# Load list of granules from dowloaded csv 
#(Those are the resulting csv from eods-api-example-generate-cloudless-mosaic Notebook)
col_list = ["typename"]
min_cloud_per_granule = pd.read_csv(r"C:\Users\Charlie.Howarth\Miniconda3\Scripts\data\min_cloud_per_granule.csv")

min_cloud_per_granule_per_orbit = pd.read_csv(r"C:\Users\Charlie.Howarth\Miniconda3\Scripts\data\min-cloud-per-granule-per-orbit.csv")

In [5]:
### USER INPUT
### Enter your token in PRD_DM = 

wps_job_request_list = [
     {'tool':'gs:Download','layer_list':min_cloud_per_granule_per_orbit["typename"],'dl_bool':True}
]

PRD_DM = "Zz1Q3arH4NroQAYKixNTw6ik6G0Reg"
ACCESS_TOKEN = PRD_DM

ENV_PRD = 'https://earthobs.defra.gov.uk'
wps_env = ENV_PRD

wps_server = wps_env + '/geoserver/ows/?access_token=' + ACCESS_TOKEN + '&SERVICE=WPS&VERSION=1.0.0'

In [6]:
# submitting the WPS request(s)

def mod_the_xml(xml_name,layer_name):
    """
    function read xml payload template and modify the payload with the config
    """
    
    with open(Path.cwd() / 'xml' / xml_name,'r') as template_xml:
        file_data = template_xml.read()
    modded_xml = file_data.replace('template_layer_name', layer_name)
    
    return modded_xml

def submit_wps_request(payload):
    """
    function submit the wps POST request
    """

    try:
        wps_submit_response = requests.post(wps_server + '&REQUEST=EXECUTE', data=payload, headers=headers, verify=True)
        print('\n' + datetime.utcnow().isoformat() + ' :: request POST url =' + wps_server + '&REQUEST=EXECUTE')
    except:
        print('wps POST was not successful')
    
    # receive status code
    timestamp_submission = datetime.utcnow()
    status_code = wps_submit_response.status_code
        
    if status_code == 200 and not wps_submit_response.text.find('ExceptionReport') > 0:
        execution_id = wps_submit_response.text.split('executionId=')[1].split('&')[0]        
        print('\n' + datetime.utcnow().isoformat() + ':: WPS job ' + execution_id + ' was successfully submitted')
        print('\n' + datetime.utcnow().isoformat() + ':: ' + str(wps_submit_response.content))
        return execution_id
    else:
        print(datetime.utcnow().isoformat() + ' :: wps request was not successful :: STATUS = ' + str(status_code))
        print(datetime.utcnow().isoformat() + ' :: response = ' + str(wps_submit_response.content))
        return None

execution_id_dict={}

for item in wps_job_request_list:
        
    for lyr in item['layer_list']:
        print('\n' + datetime.utcnow().isoformat() + ' :: tool=' + item['tool'] + ', lyr=' + lyr + ', download=' + str(item['dl_bool']))
        modded_xml = mod_the_xml(wps_test_config[item['tool']]['template_xml'],lyr)
        execution_id = submit_wps_request(modded_xml)
        execution_id_dict.update({execution_id:lyr})
        submission_time = datetime.utcnow()

print('\n' + datetime.utcnow().isoformat() + ' :: A TOTAL OF ' + str(len(execution_id_dict)) +  ' WPS REQUESTS SUBMITTED')

print('\n' + datetime.utcnow().isoformat() + ' :: The ExecutionIDs are :: ' + str(execution_id_dict.keys()))

print('\n' + datetime.utcnow().isoformat() + ' :: WPS Check Executions URL is ' + wps_server + '&REQUEST=GetExecutions')


2021-07-13T14:44:54.129353 :: tool=gs:Download, lyr=geonode:S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref, download=True

2021-07-13T14:44:55.576114 :: request POST url =https://earthobs.defra.gov.uk/geoserver/ows/?access_token=Zz1Q3arH4NroQAYKixNTw6ik6G0Reg&SERVICE=WPS&VERSION=1.0.0&REQUEST=EXECUTE

2021-07-13T14:44:55.576688:: WPS job ae1cb4d6-5698-4d21-b5d3-4d7d26190dde was successfully submitted

2021-07-13T14:44:55.576688:: b'<?xml version="1.0" encoding="UTF-8"?><wps:ExecuteResponse xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:xlink="http://www.w3.org/1999/xlink" xml:lang="en" service="WPS" serviceInstance="https://earthobs.defra.gov.uk/geoserver/ows?access_token=Zz1Q3arH4NroQAYKixNTw6ik6G0Reg&amp;" statusLocation="https://earthobs.defra.gov.uk/geoserver/ows?service=WPS&amp;version=1.0.0&amp;request=GetExecutionStatus&amp;executionId=ae1cb4d6-5698-4d21

In [7]:
# poll the status page and wait until all jobs are completed

# create output directory and log file
Path(Path.cwd() / 'output').mkdir(parents=True, exist_ok=True)
my_path = Path.cwd() / 'output' / str(submission_time.strftime(output_fmt) + '-' + RUNNAME)
my_path.mkdir(parents=True, exist_ok=True)
log_file_name = my_path / str(submission_time.strftime(output_fmt) + '-' + RUNNAME + '-wps-log.csv')
poll_frequency_sec = 60
frames = []

# poll api for 1 hour unless all jobs submitted are NOT STATUS=RUNNING
for i in range(1,3600):
    
    check_time = datetime.utcnow()
    if i == 1:
        print('\n#' + format(i, '03') + ' of #n   :: ' + submission_time.isoformat() + ' :: Logging file: ' + str(log_file_name))
    
    print('\n#' + format(i, '03') + ' of #n   :: ' + check_time.isoformat() + ' :: Poll Time')
    
    start_index = 0
    matching_of_dicts = []
    for page_counter in range(0,1000,10):

        # constrcut the paging URL and make the request
        all_status_url = wps_server + '&REQUEST=GetExecutions&startIndex=' + str(page_counter)

        r = requests.get(all_status_url,headers=headers)

        # parse the xml to an ordered dict using 3rd party imported module xmltodict
        d = xmltodict.parse(r.content)

        # grab the dicts of 'wps:ExecuteResponse' and add to a list
        jobs_list = [value for value in d['wps:GetExecutionsResponse']['wps:ExecuteResponse']]

        # add this to a master list
        matching_of_dicts.append(jobs_list)

        # if there is no 'next' attribute, then you're on the last xml page so break out the loop
        if not any('@next' == key for key in list(d['wps:GetExecutionsResponse'].keys())):
            break

    num_of_xml_pages = int(page_counter / 10) + 1
    duration_to_parse_xml_dt = datetime.utcnow() - check_time
    duration_to_parse_xml_sec = round(duration_to_parse_xml_dt.total_seconds(),0)
    print('#' + format(i, '03') + ' of #n   :: ' + datetime.utcnow().isoformat() + ' :: ' + str(num_of_xml_pages) + ' xml pages parsed in ' + str(duration_to_parse_xml_sec) + ' seconds')
            
    # now flatten the list into all responses
    response_list = [item['wps:Status'] for sublist in matching_of_dicts for item in sublist]

    # parse dict to pandas dataframe and set the index
    df_temp = pd.DataFrame.from_dict(response_list)
    df = df_temp.set_index('wps:JobID')

    # filter df on current job_ids
    filter_df  = df[df.index.isin(execution_id_dict.keys())]
            
    # add "check time" as column
    filter_df.loc[:,'check_time'] = check_time.isoformat()
    
    # append on some extra info to the dataframe using the index
    for index, row in filter_df.iterrows():
        filter_df.loc[index,'layer_name'] = execution_id_dict[index]
        filter_df.loc[index,'dl_url'] = wps_server + '&REQUEST=GetExecutionResult&EXECUTIONID='  + index + '&outputId=' + wps_test_config['gs:Download']['output'] + '&mimetype=' + wps_test_config['gs:Download']['mimetype']
    
    # sort out the indices and concatenate dataframes together and output to csv
    filter_df.reset_index(inplace=True)
    filter_df.set_index(['wps:JobID','check_time'],inplace=True)
    frames.append(filter_df)
    rolling_merged_df = pd.concat(frames)
    rolling_merged_df.to_csv(log_file_name)
        
    # if NO running processes, then break, otherwise loop again
    if not any(filter_df['wps:Status'] == 'RUNNING'):
        break
    print('#' + format(i, '03') + ' of #n   :: ' + datetime.utcnow().isoformat() + ' :: Jobs are still "STATUS=RUNNING"...script will poll again in ' + str(poll_frequency_sec) + 'sec time ...')
    
    time.sleep(poll_frequency_sec)
    
print('\n#' + format(i, '03') + ' of #' + format(i, '03') + ' :: ' + datetime.utcnow().isoformat() + ' :: Script has finished')


#001 of #n   :: 2021-07-13T14:44:56.661696 :: Logging file: C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\20210713T144456-APITEST130721-wps-log.csv

#001 of #n   :: 2021-07-13T14:45:11.124166 :: Poll Time
#001 of #n   :: 2021-07-13T14:45:14.593390 :: 3 xml pages parsed in 3.0 seconds
#001 of #n   :: 2021-07-13T14:45:14.623392 :: Jobs are still "STATUS=RUNNING"...script will poll again in 60sec time ...


C:\Users\Charlie.Howarth\Miniconda3\envs\eods\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\Charlie.Howarth\Miniconda3\envs\eods\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\Charlie.Howarth\Miniconda3\envs\eods\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic


#002 of #n   :: 2021-07-13T14:46:14.627219 :: Poll Time
#002 of #n   :: 2021-07-13T14:46:18.131404 :: 3 xml pages parsed in 4.0 seconds
#002 of #n   :: 2021-07-13T14:46:18.154407 :: Jobs are still "STATUS=RUNNING"...script will poll again in 60sec time ...

#003 of #n   :: 2021-07-13T14:47:18.158603 :: Poll Time
#003 of #n   :: 2021-07-13T14:47:21.313780 :: 3 xml pages parsed in 3.0 seconds
#003 of #n   :: 2021-07-13T14:47:21.335779 :: Jobs are still "STATUS=RUNNING"...script will poll again in 60sec time ...

#004 of #n   :: 2021-07-13T14:48:21.340999 :: Poll Time
#004 of #n   :: 2021-07-13T14:48:24.516187 :: 3 xml pages parsed in 3.0 seconds
#004 of #n   :: 2021-07-13T14:48:24.538155 :: Jobs are still "STATUS=RUNNING"...script will poll again in 60sec time ...

#005 of #n   :: 2021-07-13T14:49:24.541347 :: Poll Time
#005 of #n   :: 2021-07-13T14:49:27.910521 :: 3 xml pages parsed in 3.0 seconds

#005 of #005 :: 2021-07-13T14:49:27.939509 :: Script has finished


In [8]:
# export summary wps jobs with "runtimes"
sumdf = rolling_merged_df.reset_index()
grouped = sumdf.groupby(['wps:JobID','wps:Status']).first()
grouped.to_csv(my_path / str(submission_time.strftime(output_fmt) + '-' + RUNNAME + '-summary-wps-log.csv'))

In [9]:
# DOWNLOAD, WRITE FILE AND RENAME FILE
for index, row in filter_df.iterrows():
    if row['wps:Status'] == 'SUCCEEDED':
        print('\n\n' + datetime.utcnow().isoformat() + ' :: DOWNLOAD START :: URL = ' + row['dl_url'])        
        response = requests.get(row['dl_url'],headers=headers);
        print(datetime.utcnow().isoformat() + ' :: DOWNLOAD COMPLETE')
        local_file_name = my_path / str(row['layer_name'].split(':')[-1] + '.zip')
        print(datetime.utcnow().isoformat() + ' :: FILE WRITE START :: FILE = ' + str(local_file_name))        
        with open(local_file_name, 'wb') as f:
            f.write(response.content);
            f.close();        
        print(datetime.utcnow().isoformat() + ' :: FILE WRITE COMPLETE')
        print(datetime.utcnow().isoformat() + ' :: EXTRACT ARCHIVE FILE STARTED')
                
        # TODO. Check if downloaded file is actually a zip file type. Handle exception for an xml error report?
        zip_ref = ZipFile(local_file_name, 'r')
        zip_ref.extractall(my_path)
        zip_ref.close()
        print(datetime.utcnow().isoformat() + ' :: EXTRACT ARCHIVE FILE COMPLETE')
        print(datetime.utcnow().isoformat() + ' :: RENAME/CLEAN UP DIRECTORY STARTED')
        for f in zip_ref.filelist:
            if f.filename.endswith(".sld"):
                Path(my_path / f.filename).unlink()
            
            # rename the uuid.tiff to layer_name.tif
            elif f.filename.endswith(".tif") or f.filename.endswith(".tiff"):
                Path(my_path / f.filename).rename(my_path / str(row['layer_name'].split(':')[-1] + '.tif'))
            else:
                raise ValueError("ERROR - some unknown file has been given:", f)
        
        # remove the zip file from disk
        Path(local_file_name).unlink()
        print(datetime.utcnow().isoformat() + ' :: RENAME/CLEAN UP DIRECTORY COMPLETE')

print('\n\n' + datetime.utcnow().isoformat() + ' :: #### SCRIPT FINISHED')



2021-07-13T14:51:18.429881 :: DOWNLOAD START :: URL = https://earthobs.defra.gov.uk/geoserver/ows/?access_token=Zz1Q3arH4NroQAYKixNTw6ik6G0Reg&SERVICE=WPS&VERSION=1.0.0&REQUEST=GetExecutionResult&EXECUTIONID=0c884b62-1769-4e3c-9f01-1392313858aa&outputId=result.zip&mimetype=application/zip
2021-07-13T14:52:54.213211 :: DOWNLOAD COMPLETE
2021-07-13T14:52:54.213211 :: FILE WRITE START :: FILE = C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.zip
2021-07-13T14:52:56.900330 :: FILE WRITE COMPLETE
2021-07-13T14:52:56.901324 :: EXTRACT ARCHIVE FILE STARTED
2021-07-13T14:53:37.442475 :: EXTRACT ARCHIVE FILE COMPLETE
2021-07-13T14:53:37.442475 :: RENAME/CLEAN UP DIRECTORY STARTED
2021-07-13T14:53:37.561479 :: RENAME/CLEAN UP DIRECTORY COMPLETE


2021-07-13T14:53:37.562479 :: DOWNLOAD START :: URL = https://earthobs.defra.gov.uk/geoserver/ows/?access_token=Zz1Q3arH4NroQAYKixNTw6ik

In [18]:
#Importing libraries for Mosaic creation
import rsgislib
import rsgislib.imageutils
import rsgislib.rastergis
import rsgislib.imagecalc
import rsgislib.imagecalc.calcindices
import rsgislib.imageutils.imagecomp
import os
import glob

In [19]:
### uesr needs to download or create refimg which is a raster with the same projection and extent as the full extent 
### of image to be combined. If using a single granule the files can be downloaded from CEDA archive.
#Setting up the folder and the tif files for the mosaic
dirpath= my_path
search_critiria = "*.tif"
q = os.path.join(dirpath,search_critiria)
S2_fps = glob.glob(q)

In [20]:
##Importing libraries for converting .tif to .kea for the Mosaic creation
import os.path
import sys

In [21]:
#Setting up the convertion 
def replaceGTIFF_kea(inputtext):
    outputtext = ""    
    for w in inputtext:
        w = w.replace("GEOTIFF","KEA")
        w = w.replace(".TIF",".kea")
        # this line should be unnecessary since Landsat MTL files use capital letters       
        # but just in case you have one that doesn't
        w = w.replace(".tif",".kea")
        outputtext += w
    return outputtex

In [22]:
# find all *.TIF files and *MTL.txt files in the current directory
directory = os.chdir(my_path)
dirFileList = os.listdir(directory)

In [23]:
# print dirFileList
tifFileList = [f for f in dirFileList if ((f[-4:]=='.TIF')or(f[-4:]=='.tif'))]
MTLFileList = [f for f in dirFileList if (f[-7:]=='MTL.txt')]

In [24]:
#output format (GDAL code)
outFormat = 'KEA'

In [27]:
# run gdal_translate on all TIFs to convert to KEA
for t in tifFileList:
    gdaltranscmd = "gdal_translate -of "+outFormat+" "+t+" "+t[:-4]+".kea"
    print (gdaltranscmd)
    os.system(gdaltranscmd)

gdal_translate -of KEA S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea
gdal_translate -of KEA S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_valid.tif S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_valid.kea
gdal_translate -of KEA S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea


In [28]:
# create a new header file referring to .kea files rather than .TIF
for m in MTLFileList:
    inputtext = file(m).readlines()
    outputtext = replaceGTIFF_kea(inputtext)
    outputfilebase = m[:-4]
    outputfile = outputfilebase + "_kea.txt"
    out = file(outputfile,"w")
    out.write(outputtext)
    out.close()

In [31]:
#Setting up the paths for finding the created .kea files
dirpath= my_path
search_critiria_kea = "*stdsref.kea"
q = os.path.join(dirpath,search_critiria_kea)
S2 = glob.glob(q)
print(q)

C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\*stdsref.kea


In [32]:
S2

['C:\\Users\\Charlie.Howarth\\Miniconda3\\Scripts\\output\\20210713T144456-APITEST130721\\S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea',
 'C:\\Users\\Charlie.Howarth\\Miniconda3\\Scripts\\output\\20210713T144456-APITEST130721\\S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea']

In [33]:
# Mosaicing all the .kea files 
#inImages = S2
#refImg = FG_Sentinel_ROI.kea
#outCompImg = CompSentinel.kea
#outMskImg = Mask.kea

#rsgislib.imageutils.imagecomp.createMaxNDVINDWIComposite(refImg, inImages, 3, 8, 9, outRefImg, outCompImg, outMskImg, tmpPath='./tmp', gdalformat='KEA', dataType=None, calcStats=True, reprojmethod='cubic', use_mode=True)

# Mosaicing all the .kea files 
inImages = q
# refImg is any auxrillary file that will have the same extent and projection as the images in S2.
# This could be a raster that you create if your covering more than 1 granule
# alternatively refImg can be downloaded from The CEDA Archive as the granule will have a valid auxrillary file. 
refImg = r"C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_valid.kea"
outCompImg = 'CompSentinel.kea'
outMskImg = 'Mask.kea'
outRefImg = 'CompRefImage.kea'

rsgislib.imageutils.imagecomp.createMaxNDVINDWIComposite(refImg, inImages, 3, 8, 9, outRefImg, outCompImg, outMskImg, tmpPath='./tmp', gdalformat='KEA', dataType=None, calcStats=True, reprojmethod='cubic')

There are 2 images with overlap to create the composite.
In Image (1):	C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea
In Image (2):	C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea
In Image (1):	C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea
In Image (2):	C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721\S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea


In [34]:
#Converting the created .kea mosaic back to .tif 
def replaceGTIFF_kea(inputtext):
    outputtext = ""    
    for w in inputtext:
        w = w.replace("KEA","GEOTIFF")
        w = w.replace(".kea",".TIF")
        # this line should be unnecessary since Landsat MTL files use capital letters       
        # but just in case you have one that doesn't
        w = w.replace(".kea",".tif")
        outputtext += w
    return outputtex

In [37]:
# find all *.kea files and *MTL.txt files in the current directory
directory = os.chdir(r"C:\Users\Charlie.Howarth\Miniconda3\Scripts\output\20210713T144456-APITEST130721")
dirFileList = os.listdir(directory)

In [38]:
# print dirFileList
tifFileList = [f for f in dirFileList if ((f[-4:]=='.KEA')or(f[-4:]=='.kea'))]
MTLFileList = [f for f in dirFileList if (f[-7:]=='MTL.txt')]

In [39]:
#output format (GDAL code)
outFormat = 'GTiff'

In [40]:
#run gdal_translate on the .KEA Mosaic  to convert to .tiff
for t in tifFileList:
    gdaltranscmd = "gdal_translate -of "+outFormat+" "+t+" "+t[:-4]+".tif"
    print (gdaltranscmd)
    os.system(gdaltranscmd)

gdal_translate -of GTiff CompRefImage.kea CompRefImage.tif
gdal_translate -of GTiff CompSentinel.kea CompSentinel.tif
gdal_translate -of GTiff Mask.kea Mask.tif
gdal_translate -of GTiff S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea S2A_20210404_lat54lon066_T30UXE_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif
gdal_translate -of GTiff S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_valid.kea S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_valid.tif
gdal_translate -of GTiff S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.kea S2A_20210531_lat54lon066_T30UXE_ORB137_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif
